In [21]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import joblib


In [3]:
# Load Data
df = pd.read_excel('data/emotion.xlsx')


In [4]:
df.head(1)

,text,emotion
0,i didnt feel humiliated,sadness


In [5]:
texts = df['text'].astype(str).values
labels = df['emotion'].astype(str).values

In [6]:
# Encode Labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
num_classes = len(set(labels_encoded))

In [7]:
# Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded = pad_sequences(sequences, padding='post', maxlen=50)

In [8]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(padded, labels_encoded, test_size=0.2, random_state=42)

In [9]:
# Model Architecture
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(10000, 32, input_length=50),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])


c:\Users\admin\Desktop\chat app\Real-Time-Emotion-Detection-Chat-App-emotion_model\env\lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [10]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Train Model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.3430 - loss: 1.6017 - val_accuracy: 0.3988 - val_loss: 1.5688
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.4103 - loss: 1.5223 - val_accuracy: 0.5156 - val_loss: 1.4365
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.5399 - loss: 1.3250 - val_accuracy: 0.5628 - val_loss: 1.1784
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.6230 - loss: 1.0079 - val_accuracy: 0.6600 - val_loss: 0.9340
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7432 - loss: 0.7306 - val_accuracy: 0.7769 - val_loss: 0.6975
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8712 - loss: 0.4930 - val_accuracy: 0.8372 - val_loss: 0.5682
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.9151 - loss: 0.3311 - val_accuracy: 0.8444 - val_loss: 0.4819
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9327 - loss: 0.2522 - val_accuracy: 0.

In [12]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")


100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8640 - loss: 0.4183
Test Accuracy: 0.8628125190734863


In [13]:

# Example Prediction
test_text = ["I am damn"]


sequence = tokenizer.texts_to_sequences(test_text)
padded_sequence = pad_sequences(sequence, maxlen=50, padding='post')

prediction = model.predict(padded_sequence)
predicted_label_index = prediction.argmax(axis=1)[0]
predicted_emotion = label_encoder.inverse_transform([predicted_label_index])[0]

print(f"Predicted emotion: {predicted_emotion}")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
Predicted emotion: sadness


In [14]:
test_text = [
    "I hate when people lie to me.",
    "Nothing excites me anymore.",
    "I'm beyond thrilled with this achievement!",
    "This is just boring and pointless.",
    "I'm feeling anxious and restless today.",
    "Such a peaceful and calm evening.",
    "Why is everyone so rude these days?",
    "I feel completely abandoned and sad.",
    "Life feels so fulfilling right now.",
    "I'm nervous but a little excited too."
]
sequences = tokenizer.texts_to_sequences(test_text)
padded_sequences = pad_sequences(sequences, maxlen=50, padding='post')

predictions = model.predict(padded_sequences)
predicted_classes = predictions.argmax(axis=1)
predicted_emotions = label_encoder.inverse_transform(predicted_classes)

for i, text in enumerate(test_text):
    print(f"Text: {text}")
    print(f"Predicted Emotion: {predicted_emotions[i]}")
    print("-" * 50)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
Text: I hate when people lie to me.
Predicted Emotion: sadness
--------------------------------------------------
Text: Nothing excites me anymore.
Predicted Emotion: sadness
--------------------------------------------------
Text: I'm beyond thrilled with this achievement!
Predicted Emotion: joy
--------------------------------------------------
Text: This is just boring and pointless.
Predicted Emotion: sadness
--------------------------------------------------
Text: I'm feeling anxious and restless today.
Predicted Emotion: fear
--------------------------------------------------
Text: Such a peaceful and calm evening.
Predicted Emotion: joy
--------------------------------------------------
Text: Why is everyone so rude these days?
Predicted Emotion: anger
--------------------------------------------------
Text: I feel completely abandoned and sad.
Predicted Emotion: sadness
--------------------------------------------------
Text: Life feels so

In [17]:
model.save("saved_model/emotion_model.keras") 

In [22]:

# Tokenizer and Encoder via joblib
joblib.dump(tokenizer, "saved_model/tokenizer.joblib")
joblib.dump(label_encoder, "saved_model/label_encoder.joblib")

['saved_model/label_encoder.joblib']